In [1]:
import numpy as np
import pandas as pd

In [2]:
congFile = '/home/hassaan/Data/TUM/2nd Year/4th Semester/IDP/Project/data/intermediate/consolidated_end116.csv'
firmFile = '/home/hassaan/Data/TUM/2nd Year/4th Semester/IDP/Project/data/intermediate/comp_firms.csv'


In [3]:
congDF = pd.read_csv(congFile)
congDF = congDF.drop(['Unnamed: 0'], axis=1)
congDF.head()

,cid,personName,year,orgid,orgname,industry,asset_type,min,max,hidemax,type,lobbies,contributes,folderOfOrigin
0,N00000939,Nadler-Jerrold,2014,D000000103,JPMorgan Chase & Co,Commercial Banks,NaN,15001,50000,0,Y,Y,NaN,cong111
1,N00000939,Nadler-Jerrold,2014,D000021792,HSBC Bank,Commercial Banks,Cash/Money Market/Savings/Checking,1001,15000,0,Y,NaN,NaN,cong111
2,N00000939,Nadler-Jerrold,2014,NaN,Congressional Federal Credit Union Account,Unknown,Cash/Money Market/Savings/Checking,1001,15000,0,Y,NaN,NaN,cong111
3,N00000939,Nadler-Jerrold,2011,D000021792,HSBC Bank,Commercial Banks,Cash/Money Market/Savings/Checking,15001,50000,0,Y,NaN,NaN,cong111
4,N00000939,Nadler-Jerrold,2011,D000000103,JPMorgan Chase & Co,Commercial Banks,NaN,1001,15000,0,Y,Y,NaN,cong111


In [4]:
firmDF = pd.read_csv(firmFile)
firmDF.head()

,gvkey,conm,conml,state
0,1000,A & E PLASTIK PAK INC,A & E Plastik Pak Inc,NaN
1,1001,A & M FOOD SERVICES INC,A & M Food Services Inc,OK
2,1002,AAI CORP,AAI Corp,MD
3,1003,A.A. IMPORTING CO INC,A.A. Importing Co Inc,MO
4,1004,AAR CORP,AAR Corp,IL


In [5]:
print(len(firmDF['gvkey'].unique()))
print(len(firmDF['conm'].unique()))
print(firmDF['conm'].describe())
print(len(firmDF))
firmDF.describe()

32813
32810
count           32813
unique          32810
top       LUMENIS LTD
freq                2
Name: conm, dtype: object
32813


,gvkey
count,32813.000000
mean,58914.747783
std,66792.343541
min,1000.000000
25%,9787.000000
50%,23959.000000
75%,115960.000000
max,319507.000000


In [6]:
firmDF[firmDF['conm']=='LUMENIS LTD']

,gvkey,conm,conml,state
14777,19845,LUMENIS LTD,Lumenis Ltd,NaN
21697,62471,LUMENIS LTD,Lumenis Ltd,NaN


In [7]:
print(congDF['cid'].unique())
print(len(congDF['cid'].unique()))
print(len(congDF['folderOfOrigin'].unique()))

['N00000939' 'N00004227' 'N00005178' ... 'N00030418' 'N99999922'
 'N00000286']
1012
12


In [8]:
print(congDF['personName'].describe())

count         245582
unique          1012
top       Kerry-John
freq            4873
Name: personName, dtype: object


In [9]:
congDF.describe()

,year,min,max,hidemax
count,245582.000000,2.455820e+05,2.455820e+05,245582.000000
mean,2009.709201,1.423359e+05,3.642206e+05,0.010819
std,3.367244,2.100153e+06,2.941882e+06,0.103451
min,2004.000000,-3.547540e+05,-3.547540e+05,0.000000
25%,2007.000000,1.001000e+03,1.500000e+04,0.000000
50%,2010.000000,1.500100e+04,3.986150e+04,0.000000
75%,2013.000000,5.000100e+04,1.000000e+05,0.000000
max,2015.000000,3.120000e+08,3.120000e+08,1.000000


### Testing string matches

#### OBSERVATIONS:
- Do the process.extract() & then run a fuzzy match on ratio and partial ratio(this checks for substring)
  for added netting

In [10]:
congOrgs = congDF['orgname'].unique()
firmOrgs = firmDF['conm'].unique()
firmOrgsLC = firmDF['conml'].unique()

In [11]:
print('unique cong orgs:', len(congOrgs))
print('unique firm orgs:', len(firmOrgs))
print('unique firm orgs lower case:', len(firmOrgsLC))

unique cong orgs: 70512
unique firm orgs: 32810
unique firm orgs lower case: 32682


In [12]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from fuzzywuzzy import utils
from fuzzywuzzy.string_processing import StringProcessor

In [13]:
searchSTR = congOrgs[5000]
print(searchSTR.lower())
print('\n')
print(process.extract(searchSTR.lower(), firmOrgs, limit=5))
print('\n')
print(process.extract(searchSTR.lower(), firmOrgsLC, limit=5))

university of new mexico rev bond


[('ALLTEL NEW YORK INC', 86), ('INVESCO BOND FUND', 86), ('DIVERSICARE CP OF AMER', 86), ('GALLERY OF HISTORY INC', 86), ('NEW AMERICAN SHOE CO', 86)]


[('Alltel New York Inc', 86), ('Invesco Bond Fund', 86), ('VanEck Vectors Emerging Markets High Yield Bond ETF', 86), ('Gallery of History Inc', 86), ('New American Shoe Co', 86)]


In [14]:
orgMatch, per = process.extractOne(searchSTR, firmOrgs)
orgMatchLC, per = process.extractOne(searchSTR, firmOrgsLC)
print(searchSTR, orgMatch ,fuzz.ratio(searchSTR, orgMatch))
print(searchSTR, orgMatchLC ,fuzz.ratio(searchSTR, orgMatchLC))

University of New Mexico Rev Bond ALLTEL NEW YORK INC 19
University of New Mexico Rev Bond Alltel New York Inc 35


In [15]:
firmDF[firmDF['conml']=='CACI International Inc.']# firmDF[firmDF['conm']=='ABVIVA INC']

,gvkey,conm,conml,state
1491,2596,CACI INTL INC -CL A,CACI International Inc.,VA


In [16]:
fuzz.partial_ratio('CACI International', 'CACI International Inc.') # fuzz.partial_ratio('Citibank Cash Reserves Account 27', 'CityBank')

100

In [17]:
def levenshteinDistance(s1, s2):
    if len(s1) > len(s2):
        s1, s2 = s2, s1

    distances = range(len(s1) + 1)
    for i2, c2 in enumerate(s2):
        distances_ = [i2+1]
        for i1, c1 in enumerate(s1):
            if c1 == c2:
                distances_.append(distances[i1])
            else:
                distances_.append(1 + min((distances[i1], distances[i1 + 1], distances_[-1])))
        distances = distances_
    return distances[-1]

def levRatio(s1, s2):
    l = levenshteinDistance(s1, s2)
    m = max(len(s1), len(s2))
    return (1 - (float(l)/float(m))) * 100

In [18]:
print(levRatio(searchSTR, orgMatch))

print(levRatio(searchSTR, orgMatchLC))

15.151515151515149
27.27272727272727


In [19]:
print(fuzz.ratio(searchSTR, orgMatch))
print(fuzz.ratio(searchSTR, orgMatchLC))

19
35


### Fuzzy Wuzzy explanations
- fuzz.ratio() -> edit distance simply (levenshtein)
- fuzz.partial_ratio() -> checks for search str to be substr 
- fuzz.token_sort_ratio() -> alphabetically sort both string tokens and then match ratio
- fuzz.token_set_ratio() -> make set(intersect) from both strings and create combinations with both & match ratio

In [20]:
import re

In [21]:
# congOrgs = sorted(congOrgs)

In [22]:
print(congOrgs)

['JPMorgan Chase & Co' 'HSBC Bank'
 'Congressional Federal Credit Union Account' ...
 'Goldman Sachs/Canadian Dollars' 'Goldman Sachs/Australian Dollars'
 'BCIP Assoc III B']


In [23]:
results = [re.search("^[0-9]", l, flags=re.IGNORECASE) for l in congOrgs]

print(len(results))
# for i in range(len(congOrgs)): 
#     print(re.search('^a', congOrgs[i], flags=re.IGNORECASE))

70512


In [24]:
# print(results)

In [25]:
# results = list(filter(None.__ne__, results))
res = [i for i in range(len(results)) if not results[i] is None]
print([congOrgs[res[i]] for i in range(len(res))])

['401k Pension Plan American Funds', '401K /Pension Plan American Funds', '7344 Gwin St/Valley Springs CA', '7344 Gwin St/Valley Spring CA', '7400 Gwin St Valley/Springs, CA', '7344 Gwin St/Valley Springs, CA', '2200 acres land/Lindsay, TX', '292 acres land/Montague County, TX', '12 acres/Dallas, TX', '167 acres/Valley View, TX', '3M Co', '500 Bank Insured Deposit', '10 oz Gold Bar', '1290 Business DEV COM OPPTYS', '1303 TACT INC Closed-End', '1272 International High 30 DVD', '535 Marina Street/Rio Grande, PR', '615 Vistamar Street/Rio Grande, PR', '535 Marina St/Rio Grande, Puerto Rico', '615 Vistamar St/Rio Grande, Puerto Rico', '2025 W St Paul/Chicago, IL', '1003 W Fulton/Chicago, IL', '2146 W Churchill/Chicago, IL', '1 Lot/Columbia, SC', '1/3 Lot/Sumter, SC', '1/2 interest in two lots', '1/3 interest in unimproved land', '1 Lot', '1/3 Lot Gift/Sumter, SC', '529C', '113 Inherited Interest of Apartment in U', '6030 Woodrun Court/Montgomery, AL', '35 Bluff Drive LLC', '18th St/Tybee, 

#### Change the entries in congOrgs[res[i]] with '% '[1] to company names on a case by case basis and insert back where ever they're used in the actual dataframe

In [26]:
congDF[congDF['orgname'] == '15% interest in Legal Press, LLC']

,cid,personName,year,orgid,orgname,industry,asset_type,min,max,hidemax,type,lobbies,contributes,folderOfOrigin
177341,N00035607,Trott-Dave,2015,NaN,"15% interest in Legal Press, LLC",NaN,Cash/Money Market/Savings/Checking,1001,15000,0,Y,NaN,NaN,cong114


In [27]:
print(len(congDF['asset_type'].unique()))
congDF['asset_type'].unique()

9


array([nan, 'Cash/Money Market/Savings/Checking', 'Brokerage, IRA, 401k',
       'Government Bond', 'Unknown',
       '"Account" or Combination of other types (I.e.Stocks & Bonds)',
       'Insurance policies & Annuities', 'Mutual Fund',
       'Note/Note Receivable'], dtype=object)

In [76]:
# REGEX patterns (https://help.libreoffice.org/Common/List_of_Regular_Expressions)
pattern = re.compile('\+?%\s')  #testing  regex (https://www.regextester.com/94730)
matches = [x for x in congOrgs if pattern.findall(x)]

In [77]:
print(matches)

['Ken-Ran Development 50% Interest Land', 'European Investment Bank Mtn 8.50000% 11', 'Citigroup Funding 3% based on Gold', '25% Interest Inherited from the Settlement of the', 'Wells Fargo Captial 7% Trust', 'ICMA-RC457 Deferred Comp Plan 100% in Vantage\nTr', 'Millco of Wilson LLC-600 Nash Street, 50% Interest', 'Aes Tr III 6.75% Conv', 'USB Capital X 6.5% Wells Fargo Cap XIV', 'Coupon Treasury Receipts INT PMT 12.5% o', 'DB 100% PPN-ABS RTN BAR S&P 500', 'UBS 100% PRAB S&P 500', 'Oakland MI Macomb 2.0% municipal bond', 'Manchester MI Cmnty 2.0% municipal bond', 'Inland Lakes MI Sch 2.25% municipal bond', 'Big Rapids MI Sch 3.25% municipal bond', '25% interest in ZLDR partnership owning 95 acres +', 'Hartford Life Insurance 75% notes', 'Gateway Inc 1.5% 12/31/09', 'Kentucky Asset/Liab 5% 7/15/06', 'Fidelity Asset Manager 50% Fund', '20% Interest Travis Webb', 'Travis Webb/20% interest', 'Fidelity UFUND 70% Equity Portfolio', 'Fidelity UFUND 100% Equity Portfolio', 'Fidelity College I

In [30]:
congDF[congDF['orgname'].isin(matches)].head()

,cid,personName,year,orgid,orgname,industry,asset_type,min,max,hidemax,type,lobbies,contributes,folderOfOrigin
3838,N00026710,Marchant-Kenny,2011,NaN,Ken-Ran Development 50% Interest Land,Unknown,NaN,50001,100000,0,Y,NaN,NaN,cong111
5479,N00006263,Simpson-Mike,2011,NaN,European Investment Bank Mtn 8.50000% 11,Unknown,NaN,28763,28763,0,Y,NaN,NaN,cong111
6611,N00012508,Carper-Tom,2012,NaN,Citigroup Funding 3% based on Gold,Unknown,Mutual Fund,15001,50000,0,Y,NaN,NaN,cong111
6738,N00012508,Carper-Tom,2011,NaN,Citigroup Funding 3% based on Gold,Unknown,Mutual Fund,15001,50000,0,Y,NaN,NaN,cong111
7498,N00012508,Carper-Tom,2010,NaN,Citigroup Funding 3% based on Gold,Unknown,Mutual Fund,50001,100000,0,Y,NaN,NaN,cong111


In [31]:
perList_sep = [s.split('% ') for s in matches]#map(lambda s: s.split('% ')[1], congDF['orgname'].values)

- It is important to note here that the '% 'tokenizer is quite confusing since the strings being tokenized are not     consistent. Some have comapny/property name after the tokenizer and some before.
- Therefore, the approach should be able to take all possible tokens (theoretically only two at max.) and check         matches

In [32]:
print(perList_sep[10]) 
# print(perList_sep[1])

['DB 100', 'PPN-ABS RTN BAR S&P 500']


In [33]:
perList_sep = []
for s in matches:
    temp = s.split('% ')
    perList_sep.append(temp[0]+temp[1])
#     if len(temp[0])>len(temp[1]) and len(temp[0].split(' '))>len(temp[1].split(' ')):
#         perList_sep.append(temp[0])
#     else:
#         perList_sep.append(temp[1])

In [34]:
matches.index('10% interest in Trott-ATA LLC')

121

In [35]:
print(perList_sep[16])

25interest in ZLDR partnership owning 95 acres +


##### Double string matching function for getting a very close match

In [36]:
print(process.extract(searchSTR, firmOrgsLC, limit=5))
print(fuzz.partial_ratio('CACI International', 'CACI International Inc'))

[('Alltel New York Inc', 86), ('Invesco Bond Fund', 86), ('VanEck Vectors Emerging Markets High Yield Bond ETF', 86), ('Gallery of History Inc', 86), ('New American Shoe Co', 86)]
100


In [37]:
def strMatch(searchVal, indexList, l1_thresh=85, l2_thresh=90, _size=5):
    matches = []
    baseList = process.extract(searchVal, indexList, limit=_size)
#     print('BASE_LIST: ',baseList)
    for _str, conf in baseList:
        if float(conf)>l1_thresh:
            match = fuzz.partial_ratio(searchVal, _str)
            if float(match)>l2_thresh:
                matches.append((_str, match))
    return baseList, matches

In [ ]:
searchSTR = 'JPMorgan 100US Treasury Securities Mon' # 'CACI International'
print(searchSTR,'\n\n')

BL, Matches = strMatch(searchSTR, firmOrgsLC)
print(BL)
print('\n')
print(Matches)
print('\n')
print('Highest Match:', Matches[0])

In [ ]:
print(fuzz.token_sort_ratio('JPMorgan 100', 'JPMorgan Chase & Co'))

In [ ]:
orgClean = []
tempDF = congDF[congDF['orgname'].isin(matches)]
for index, row in tempDF.iterrows():
    temp = row['orgname'].split('% ')
    orgClean.append(temp[0]+temp[1])
tempDF['orgClean'] = orgClean

In [ ]:
# matchIDX = []
# blIDX = []
# for index, row in congDF.iterrows():
#     BL, Matches = strMatch(row['orgname'], firmOrgsLC)
#     if Matches:
#         matchIDX.append(Matches[0])
#         blIDX.append(BL)
#     else:
#         matchIDX.append(np.nan)
#         blIDX.append(BL)

In [ ]:
# congDF['orgMatch'] = matchIDX
# congDF['orgBestOptions'] = blIDX

# tempDF.head(10)

In [ ]:
congDF.tail()

In [ ]:
congDF.to_csv('consolidatedMatched.csv')

### creating unique matches to reduce compute time

In [ ]:
total_congOrgs = len(congOrgs)
i=0
match_dict = {}
baseList_dict = {}
for congFirm in congOrgs:
    BL, Matches = strMatch(congFirm, firmOrgsLC)
    
    baseList_dict[congFirm] = BL
    if Matches:
        match_dict[congFirm] = Matches[0]
        
    else:
        match_dict[congFirm] = np.nan
    
    #counter
    i+=1
    print((float(i)/total_congOrgs)*100)

In [45]:
print(len(match_dict))
print(len(baseList_dict))

70512
70512


In [46]:
set(match_dict.keys()) == set(baseList_dict.keys())

True

In [67]:
matchedDF = pd.DataFrame(match_dict.items(), columns=['congOrgs', 'match'])
baseListDF = pd.DataFrame(baseList_dict.items(), columns=['congOrgs', 'baseList'])

In [68]:
matchedDF.head()

,congOrgs,match
0,JPMorgan Chase & Co,"(JPMorgan Chase & Co, 100)"
1,HSBC Bank,NaN
2,Congressional Federal Credit Union Account,NaN
3,Chase Bank,"(Chevy Chase Bank FSB, 100)"
4,Citibank Accounts,NaN


In [69]:
baseListDF.head()

,congOrgs,baseList
0,JPMorgan Chase & Co,"[(JPMorgan Chase & Co, 100), (Acushnet Co, 86)..."
1,HSBC Bank,"[(American Bank Note Co, 86), (Bank of Hawaii ..."
2,Congressional Federal Credit Union Account,"[(Aetna Business Credit Inc, 86), (Allis-Chalm..."
3,Chase Bank,"[(Chevy Chase Bank FSB, 90), (American Bank No..."
4,Citibank Accounts,"[(CityBank, 79), (Citation Cos, 62), (Key Co, ..."


In [70]:
matchedDF.to_csv('matchedDF.csv')
baseListDF.to_csv('baseListDF.csv')

#### Weird company names

In [73]:
print('unique companies: ', len(congOrgs))

unique companies:  70512


In [74]:
pattern = re.compile('\+?%\s')  #testing  regex (https://www.regextester.com/94730)
matches = [x for x in congOrgs if pattern.findall(x)]

print('companies with "% " sub-string: ', len(matches))

companies with "% " sub-string:  171


In [75]:
pattern = re.compile('^[0-9]')  #testing  regex (https://www.regextester.com/94730)
matches = [x for x in congOrgs if pattern.findall(x)]

print('unique companies starting with number: ', len(matches))

unique companies starting with number:  2203
